# **Installalling / Importing libraries**

In [ ]:
# (Colab) Install any missing libs - safe to run even if already available
!pip install openpyxl plotly --quiet

# Imports
import os
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Optional: display plots inline nicely
%matplotlib inline

# Output folder for generated files (Colab's /content or mount Drive)
OUT_DIR = "/content/smart_report"
os.makedirs(OUT_DIR, exist_ok=True)
print("Output folder:", OUT_DIR)


# **Uploading Excel (.xlsx)**

In [ ]:
from google.colab import files
uploaded = files.upload()  # Select your Excel file when prompted
# After upload, get filename:
fname = next(iter(uploaded.keys()))
print("Uploaded:", fname)

In [ ]:
import pandas as pd

xls = pd.ExcelFile(fname)
print("Sheet names:", xls.sheet_names)


# **Checking the file.**

In [ ]:
df = pd.read_excel(fname)
df.head()


# **Loading the Excel sheets into DataFrames**

In [ ]:
# If you uploaded file, use fname. Otherwise use sample_path from above.
excel_path = fname if 'fname' in globals() else sample_path
print("Reading:", excel_path)

df_audit = pd.read_excel(excel_path, sheet_name='audit_findings')
df_seo   = pd.read_excel(excel_path, sheet_name='seo_performance')


# **Cleaning & normalizing data (light, robust cleaning)**


In [ ]:
def clean_audit(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    # strip strings
    for c in ['Department','Clause','Finding','Severity','Notes']:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
    df = df.dropna(subset=['Date'])  # remove rows without dates
    return df

def clean_seo(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce').dt.date
    for c in ['Page','Keyword']:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
    # numeric safety
    for c in ['Impressions','Clicks','AvgPosition']:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)
    return df

df_audit = clean_audit(df_audit)
df_seo = clean_seo(df_seo)

print("Audit rows:", len(df_audit), "SEO rows:", len(df_seo))


# **Small derived metrics (risk score & aggregates)**



> Creating the main summaries the report will show.



In [ ]:
# Simple severity to score map
sev_map = {'Low':1, 'Medium':2, 'High':3}
df_audit['SeverityScore'] = df_audit['Severity'].map(sev_map).fillna(1).astype(int)

# Aggregations for Audit Data
audit_by_dept = df_audit.groupby('Department').size().reset_index(name='FindingsCount').sort_values('FindingsCount', ascending=False)
dept_avg_sev = df_audit.groupby('Department')['SeverityScore'].mean().reset_index().sort_values('SeverityScore', ascending=False)

# Aggregations for SEO Data (Base metrics)
seo_top_pages = df_seo.groupby('Page')[['Impressions','Clicks']].sum().reset_index().sort_values('Clicks',ascending=False)
seo_top_keywords = df_seo.groupby('Keyword')[['Impressions','Clicks']].sum().reset_index().sort_values('Clicks',ascending=False)
seo_time = df_seo.groupby('Date')[['Clicks','Impressions']].sum().reset_index().sort_values('Date')


# --- NEW LOGIC: Keyword Suggestion (CTR and Filtering) ---

# 1. Compute total metrics per keyword (used for CTR calculation)
keyword_stats = (
    df_seo.groupby('Keyword', as_index=False)
    .agg({'Impressions': 'sum', 'Clicks': 'sum'})
)

# 2. Calculate overall CTR
keyword_stats['CTR'] = keyword_stats['Clicks'] / keyword_stats['Impressions']
keyword_stats['CTR'] = keyword_stats['CTR'].fillna(0) # Handle division by zero

# 3. Find high-impression but low-CTR keywords (suggested for optimization)
impression_threshold = keyword_stats['Impressions'].quantile(0.75)
low_ctr_threshold = keyword_stats['CTR'].quantile(0.25)

suggested_keywords = keyword_stats[
    (keyword_stats['Impressions'] >= impression_threshold) &
    (keyword_stats['CTR'] <= low_ctr_threshold)
].sort_values(by='Impressions', ascending=False)
# --- END NEW LOGIC ---


# Show quick preview (No longer duplicated)
display(audit_by_dept.head(), dept_avg_sev.head(), seo_top_pages.head())

# **Adding Keyword Top-N Suggestions Function**

In [ ]:
df_seo.rename(columns={
    'Search term': 'Keyword',
    'Click Count': 'Clicks',
    'Impression Count': 'Impressions'
}, inplace=True)

# Compute CTR (Click-Through Rate)
df_seo['CTR'] = df_seo['Clicks'] / df_seo['Impressions']
df_seo['CTR'] = df_seo['CTR'].fillna(0)  # handle division by zero


#Aggregate by keyword
keyword_stats = (
    df_seo.groupby('Keyword', as_index=False)
    .agg({'Impressions': 'sum', 'Clicks': 'sum'})
)
keyword_stats['CTR'] = keyword_stats['Clicks'] / keyword_stats['Impressions']


#Find high-impression but low-CTR keywords
impression_threshold = keyword_stats['Impressions'].quantile(0.75)
low_ctr_threshold = keyword_stats['CTR'].quantile(0.25)

suggested_keywords = keyword_stats[
    (keyword_stats['Impressions'] >= impression_threshold) &
    (keyword_stats['CTR'] <= low_ctr_threshold)
].sort_values(by='Impressions', ascending=False)

#Show Top-N keywords
top_n = 10
print("Top Keyword Suggestions:")
display(suggested_keywords.head(top_n))



# **Quick plots (matplotlib) — keeping visuals simple and clear**

In [ ]:
plt.figure(figsize=(8,4))
plt.bar(audit_by_dept['Department'], audit_by_dept['FindingsCount'])
plt.title('Audit Findings by Department')
plt.xlabel('Department'); plt.ylabel('Findings Count')
plt.tight_layout(); plt.show()

plt.figure(figsize=(6,6))
plt.pie(df_audit['Severity'].value_counts(), labels=df_audit['Severity'].value_counts().index, autopct='%1.1f%%')
plt.title('Findings by Severity')
plt.show()

plt.figure(figsize=(8,4))
plt.plot(pd.to_datetime(seo_time['Date']), seo_time['Clicks'], marker='o')
plt.title('SEO Clicks Over Time')
plt.xlabel('Date'); plt.ylabel('Clicks')
plt.xticks(rotation=30); plt.tight_layout(); plt.show()


# **Exporting summary CSVs**

In [ ]:
audit_by_dept.to_csv(os.path.join(OUT_DIR, 'audit_by_department.csv'), index=False)
dept_avg_sev.to_csv(os.path.join(OUT_DIR, 'department_avg_severity.csv'), index=False)
seo_top_pages.to_csv(os.path.join(OUT_DIR, 'seo_top_pages.csv'), index=False)
seo_top_keywords.to_csv(os.path.join(OUT_DIR, 'seo_top_keywords.csv'), index=False)
seo_time.to_csv(os.path.join(OUT_DIR, 'seo_time_series.csv'), index=False)
print("CSV summaries saved to:", OUT_DIR)


# **Exporting everything to a single PDF report (one-page per plot + title page)**




> With simple textual summary page into the PDF



In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
import os
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd # Ensure pandas is available here

pdf_file = os.path.join(OUT_DIR, 'smart_report.pdf')

# --- Logic for the FINAL Summary Text (must run first to define insights) ---

# Find the keyword insight (assuming suggested_keywords is defined in Cell 5)
keyword_insight = "- No keywords met the High Impressions / Low CTR criteria."
if 'suggested_keywords' in globals() and not suggested_keywords.empty:
    # Find the keyword with the highest impressions among the suggestions
    top_suggestion = suggested_keywords.iloc[0]
    keyword_insight = (f"- Top keyword suggestion: {top_suggestion['Keyword']} "
                       f"({int(top_suggestion['Impressions'])} impressions / {top_suggestion['CTR']:.2f}% CTR)")

# Construct the full summary text
summary_text = [
    f"Report generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
    "",
    "Top insights:",
    f"- Department with most findings: {audit_by_dept.iloc[0]['Department']} ({int(audit_by_dept.iloc[0]['FindingsCount'])} findings)" if not audit_by_dept.empty else "- No audit data",
    f"- Top SEO page by clicks: {seo_top_pages.iloc[0]['Page']} ({int(seo_top_pages.iloc[0]['Clicks'])} clicks)" if not seo_top_pages.empty else "- No SEO data",
    keyword_insight
]

# --- PDF GENERATION (Combines all pages into one file) ---

with PdfPages(pdf_file) as pdf:

    # 1. Title page
    fig = plt.figure(figsize=(8.27, 11.69))
    fig.text(0.5, 0.6, "Smart Report Generator", ha='center', fontsize=22)
    fig.text(0.5, 0.5, f"Auto-generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
             ha='center', fontsize=10)
    pdf.savefig(fig)
    plt.close(fig)

    # 2. Audit Findings by Department
    fig, ax = plt.subplots(figsize=(8, 4))
    ax.bar(audit_by_dept['Department'], audit_by_dept['FindingsCount'], color='steelblue')
    ax.set_title('Audit Findings by Department')
    ax.set_ylabel('Findings')
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

    # 3. Findings by Severity
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.pie(df_audit['Severity'].value_counts(),
           labels=df_audit['Severity'].value_counts().index,
           autopct='%1.1f%%')
    ax.set_title('Findings by Severity')
    pdf.savefig(fig)
    plt.close(fig)

    # 4. SEO Clicks Over Time (Restored from previous step)
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(pd.to_datetime(seo_time['Date']), seo_time['Clicks'], marker='o', color='forestgreen')
    ax.set_title('SEO Clicks Over Time')
    ax.set_xlabel('Date')
    ax.set_ylabel('Clicks')
    plt.xticks(rotation=30)
    plt.tight_layout()
    pdf.savefig(fig)
    plt.close(fig)

    # 5. Keyword Suggestions (The new chart)
    fig, ax = plt.subplots(figsize=(8, 4))
    top_keywords = suggested_keywords.head(10)
    if not top_keywords.empty:
        ax.barh(top_keywords['Keyword'], top_keywords['Impressions'], color='skyblue')
        ax.set_xlabel('Impressions')
        ax.set_title('Top 10 Keywords (High Impressions, Low CTR)')
        ax.invert_yaxis()
    else:
        ax.text(0.5, 0.5, "No keywords met the High Impressions / Low CTR criteria.",
                ha='center', va='center', fontsize=12)
        ax.set_title('Keyword Suggestions')
        ax.axis('off')

    fig.text(0.5, 0.05,
             "These keywords have strong reach but low CTR — potential for optimisation.",
             ha='center', fontsize=8)

    plt.tight_layout(rect=[0, 0.1, 1, 1])
    pdf.savefig(fig)
    plt.close(fig)

    # 6. Textual Summary Page (The old Cell 9 content)
    fig = plt.figure(figsize=(8.27, 11.69))
    fig.text(0.05, 0.95, "\n".join(summary_text), fontsize=11, va='top', family='monospace')
    pdf.savefig(fig)
    plt.close(fig)

print("✅ PDF report saved successfully:", pdf_file)

# **Wraping into functions & run pipeline**

In [ ]:
def run_report(excel_path, out_dir=OUT_DIR):
    # 1) load
    df_audit = pd.read_excel(excel_path, sheet_name='audit_findings')
    df_seo = pd.read_excel(excel_path, sheet_name='seo_performance')
    # 2) clean
    df_audit = clean_audit(df_audit)
    df_seo = clean_seo(df_seo)
    # 3) compute summaries (same code as before)
    # ... (copy the aggregation code from Cell 5 here)
    # 4) save CSVs and PDF (call export code from Cells 7-9)
    print("run_report completed for", excel_path)

# Example call:
# run_report(excel_path)


# **Downloading final pdf report in my computer.**






> To download it to your computer, run this in a new cell otherwise **file is saved inside Colab’s temporary runtime storage**





In [ ]:
from google.colab import files
files.download('/content/smart_report/smart_report.pdf')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Mount Google Drive: if you want to save outputs to Drive permanently**



> For this you have to connect your drive with colab.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
OUT_DIR = '/content/drive/MyDrive/smart_report'
os.makedirs(OUT_DIR, exist_ok=True)
